In [1]:
#Import Packages
import pandas as pd
import datetime
from ErcotApi import ercot_connection
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


DA and RT Prices

In [6]:
#ERCOT Prices Calls
dtfrom = datetime.date.today() + datetime.timedelta(days=-6) 
dtto = datetime.date.today() + datetime.timedelta(days=-6) 

#hubs_lz = ["HB_NORTH", "HB_HOUSTON", "HB_SOUTH", "HB_WEST", "LZ_NORTH", "LZ_HOUSTON", "LZ_SOUTH", "LZ_WEST"]
hubs_lz = ["HB_NORTH"]

rtm_url = ["np6-905-cd/spp_node_zone_hub?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPointType=HU&size=1000",
           "np6-905-cd/spp_node_zone_hub?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPointType=LZ&size=1000"]

#rtm_url = ["np6-905-cd/spp_node_zone_hub?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPointType=HU&size=1000"]

da_url = ["np4-190-cd/dam_stlmnt_pnt_prices?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPoint=HB_NORTH&size=1000"]


rtm_hublz = ercot_connection(rtm_url)
da_hublz = ercot_connection(da_url)

#ERC Prices Data Clean
def ercot_prices(price_data, market):
    prices = pd.DataFrame()
    for df in price_data:     
        
        if  market.upper() == "RT":
            df =df[df["settlementPoint"]!= "HB_PAN"]
            spp = df
            #spp  = df.groupby(["deliveryDate", "deliveryHour", "settlementPoint"])["settlementPointPrice"].mean().reset_index()
            spp["market"] = "RT"
        else:
            df["market"] = "DA"
            spp = df.rename({"hourEnding":"deliveryHour"}, axis=1)
            spp["deliveryHour"] = pd.to_numeric(spp["deliveryHour"].str.split(":").str[0])
            spp = spp.drop('DSTFlag', axis=1)
            
        deliveryHour = [datetime.time(num-1).strftime("%H:00:00") for num in spp["deliveryHour"]]
        spp["DateTime"] = pd.to_datetime(spp["deliveryDate"] + " "+ deliveryHour, format = "%Y-%m-%d %H:%M:%S")
        spp["type"] = spp["settlementPoint"].str.split("_").str[0]

        prices = pd.concat([prices, spp], ignore_index=True)
    return(prices)
rt_hub_lz = ercot_prices(rtm_hublz, "RT")


In [6]:
#Prices Summary
#ERCOT Prices Calls for Hubs
def ercot_price_summary(from_days, to_days):
    dtfrom = datetime.date.today() + datetime.timedelta(days=from_days) 
    dtto = datetime.date.today() + datetime.timedelta(days=to_days) 

    rtm_url = ["np6-905-cd/spp_node_zone_hub?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPointType=HU&size=15000"]

    #rtm_url = ["np6-905-cd/spp_node_zone_hub?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPointType=HU&size=1000"]

    da_url = ["np4-190-cd/dam_stlmnt_pnt_prices?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPoint=HB_NORTH&size=10000",
            "np4-190-cd/dam_stlmnt_pnt_prices?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPoint=HB_HOUSTON&size=10000",
            "np4-190-cd/dam_stlmnt_pnt_prices?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPoint=HB_SOUTH&size=10000",
            "np4-190-cd/dam_stlmnt_pnt_prices?deliveryDateFrom="+str(dtfrom)+"&deliveryDateTo="+str(dtto)+"&settlementPoint=HB_WEST&size=10000"]



    rt_hub_lz = ercot_prices(ercot_connection(rtm_url), "RT")
    da_hub_lz = ercot_prices(ercot_connection(da_url), "DA")
    realtime_hub = rt_hub_lz.pivot(index=["deliveryDate", "deliveryHour"], columns="settlementPoint", values="settlementPointPrice").rename_axis(None, axis=1).reset_index()
    dayahead_hub = da_hub_lz.pivot(index=["deliveryDate", "deliveryHour"], columns="settlementPoint", values="settlementPointPrice").rename_axis(None, axis=1).reset_index()
    realtime_hub = realtime_hub.rename({'deliveryHour':'Hour'}, axis=1)
    dayahead_hub = dayahead_hub.rename({'deliveryHour':'Hour'}, axis=1)
    dart_spreads = dayahead_hub.iloc[:,2:6] - realtime_hub.iloc[:,2:6]  
    dart_spreads["Date"] = realtime_hub["deliveryDate"]
    dart_spreads["Hour"] = realtime_hub["Hour"]
    #return(realtime_hub, dayahead_hub)
    return(realtime_hub, dayahead_hub, dart_spreads)

#calculatting average for days > 0
def group_means(real_time, day_ahead, day_real,days):
        
    dtfilter = datetime.date.today() + datetime.timedelta(days=-days) 
    realtm_hub = real_time[real_time["deliveryDate"] >= str(dtfilter)].groupby("Hour")[["HB_NORTH","HB_HOUSTON", "HB_SOUTH", "HB_WEST"]].mean().round(2).reset_index()
    dayahd_hub = day_ahead[day_ahead["deliveryDate"] >= str(dtfilter)].groupby("Hour")[["HB_NORTH","HB_HOUSTON", "HB_SOUTH", "HB_WEST"]].mean().round(2).reset_index()
    dt_spreads = day_real[day_real["Date"] >= str(dtfilter)].groupby("Hour")[["HB_NORTH","HB_HOUSTON", "HB_SOUTH", "HB_WEST"]].mean().round(2).reset_index() 
    return(realtm_hub, dayahd_hub, dt_spreads)

#Table formating function
def fig_tables(rt,da,dart):
    fig = make_subplots(rows=1, cols=3,
                        specs=[[{'type':'table'},{'type':'table'},{'type':'table'}]], 
                        horizontal_spacing=0.05,
                        subplot_titles=("Day Ahead", "Real Time", "DART Spreads"))

    fig.add_trace(go.Table(header=dict(values=list(da.columns)), 
                        cells = dict(values= da.transpose().values.tolist())),
                        row=1, 
                        col=1)
    fig.add_trace(go.Table(header=dict(values=list(rt.columns)), 
                        cells = dict(values= rt.transpose().values.tolist())), 
                        row=1,
                        col=2)
    fig.add_trace(go.Table(header=dict(values=list(dart.columns)), 
                        cells = dict(values= dart.transpose().values.tolist())), 
                        row=1,
                        col=3)

    fig.update_layout(
        width = 1500,
        height = 570,
        margin = dict(l=20,r=20, t=50,b=10),
    
    )
    fig.update_annotations(xshift=-170)
    return(fig)


realtime, dayahed, darts = ercot_price_summary(-30, -1)


## Current Day : 

In [10]:

price_dates = datetime.datetime.today() + datetime.timedelta(days=0)
date_today = datetime.datetime.today()
date_delta = price_dates - date_today
realtime_0, dayahed_0, darts_0 = ercot_price_summary(date_delta.days, date_delta.days)
rt_0, da_0, dart_0 = group_means(realtime_0, dayahed_0, darts_0,-date_delta.days)

fig_tables(rt_0, da_0, dart_0).update_layout(title= "Prices for " + str(price_dates.date())).show()